# Food101 Classification with SVM

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import os
import cv2
import pickle
import csv

from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2grey
from skimage import data, exposure
from skimage.data import camera

# Library for scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Library for LIBSVM
from libsvm.svmutil import *
from itertools import combinations

# Library fror tensorflow and keras
import tensorflow as tf
import tensorflow.keras
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import imagenet_utils
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import h5py

/home/insignite/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/insignite/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/insignite/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/insignite/miniconda3/lib/python3.7/site-packages/tensorflow/pyth

## Variable declaration

In [2]:
# 101 is main dataset, 102 is for testing with smaller dataset
data_dir = "/mnt/c/Users/nhmin/Downloads/food-101"
project_dir = os.getcwd()
model_dir = "/mnt/c/Users/nhmin/Downloads"
# Add to dictionary if want more class
class_label = {"pad_thai" : 0, "pho" : 1, "ramen" : 2, "spaghetti_bolognese" : 3, "spaghetti_carbonara" : 4}
high_params = [0,2,0.5]

## Ultilities functions

In [ ]:
# Load_json function use to read in train.json and test.json
# and only take data according to class_label variable
def load_json(path):
    final_data = dict()
    # Load in json file to create dictionary: key = class label; value = file path
    with open(path, 'r') as file:
        data = json.load(file)
    # Only get information from needed class
    for label in class_label:
        final_data.update({label : data.get(label)})
    return final_data


# Return np img size 227x227
def get_image(path):
    # image resize to 384x384
    img = Image.open(path + ".jpg")
    resized_image = img.resize((227,227), Image.ANTIALIAS)
    return np.array(resized_image)



# Return class label (key) according to values
# SVM return label of number so this function return
# the corresponding label
def get_key(val):
    for key, value in class_label.items():
        if(val == value): return key
    return None



# Make a dataframe have all image name and its label
def create_data(json_data):
    # Create train data
    file_names_list = []
    label_list = []
    for label in json_data.keys():
        #file_names = os.listdir(data_dir + '/images/' + label)
        file_names = json_data.get(label)
        for file in file_names:
            file = file.split('/')[1] + '.jpg'
            file_names_list.append(file)
            label_list.append(class_label.get(label))
    # Create dataframe
    data_df = pd.DataFrame({
        'filename' : file_names_list,
        'label' : label_list
    })
    return data_df

## Grid Searching

In [ ]:
# Write accuracy record to csv
def write_record_txt(path):
    kernel = []
    gamma = []
    cost = []
    acc = []
    kernel_dict = {'0' :'Linear', '1': 'Polynomial', '2' : 'RBF' }
    with open(path, 'r') as f:
        for line in f:
            values = line.split(' : ')
            params = values[0].split(',')
            kernel.append(kernel_dict.get(params[0][1:]))
            gamma.append(params[1])
            cost.append(params[2][:-1])
            acc.append(values[1][:-2])
    data_df = pd.DataFrame({
        'Kernel' : kernel,
        'Gamma' : gamma,
        'Cost' : cost,
        'Accuracy' : acc
    })
    data_df.to_csv('grid_search_record.csv')

def grid_search(train_df, train_feature):
    # Grid Search
    kernel = [0,1,2]
    cost = [2**(-5),2**(-3),2**(-1),1,2,2**3,2**5,2**7,2**9,2**11,2**13,2**15]
    gamma = [2**(-15),2**(-13),2**(-11),2**(-9),2**(-7),2**(-5),2**(-3),2**(-1),1,2,2**3]
    # Create all the combination O(n^3)
    combine_list = [(k,g,c) for k in kernel for g in gamma for c in cost]
    accuracy_record = {}
    highest_acc = -1.0
    highest_params = (0,0,0)
    for i in range(0, len(combine_list)):
        train_val_acc = svm_train(train_df['label'].tolist(), train_feature, '-s 0 -t %s -v 10 -g %s -c %s -b 1' % combine_list[i])
        if(train_model > highest_acc):
            highest_acc = train_model
            highest_params = combine_list[i]
        accuracy_record[combine_list[i]] = train_model
    print('Highest Accuracy and Params: ')
    print('(kernel, gamma, cost) : ',highest_params, ':', highest_acc)
    write_record_txt(model_dir + '/grid_search_record.txt')
    return highest_params

## Histogram Oriented of Gradients (HOG) approach

### Data processing

In [ ]:
# Taking in an impage, perform HOG conversion
# input: img path     output: nparray of HOG img
def HOG_image(path):
    np_image = load_img(path, target_size=(227,227))
    # given 32x32 cell
    image_feature, image_hog = hog(np_image, orientations=8, pixels_per_cell=(8, 8),
        cells_per_block=(8, 8), block_norm = 'L2-Hys', visualize=True, multichannel=True)
    return np.array(image_feature)

# PCA to reduce the number of feature 
def pca_transform(image):
    ss = StandardScaler()
    image_ss = ss.fit_transform(image)
    # Keep 90% of variance
    pca = PCA(0.9)
    image_pca = pca.fit_transform(image_ss)
    return mage_pca

# return a list of HOG converted img
# input: train_df      output: list of HOG arrays
def HOG_training(train_df):
    train_data = []
    size = len(train_df['filename'])
    for i in range(0, size):
        image = data_df['filename'][i]
        label = data_df['label'][i]
        path = data_dir + '/images/' + get_key(label) + '/' + image
        HOG_data = HOG_image(path)
        # this is where PCA apply. take off comment line if needed.
        # but i believe PCA return an unequal list size on every image
        #train_data.append(pca_transform(HOG_data))
        train_data.append(HOG_data)
    return train_data

### Training

In [ ]:
# load json file
train_json = load_json(data_dir + '/meta/train.json')
# Create dataframe for train
train_df = create_data(train_json)

# Convert img to HOG nparray
hog_train_feature = HOG_training(train_df)

#====Uncoment for Grid search==========
#high_params = grid_search(train_df, hog_train_feature)
#===========End block==================

# b = 1 will give prob est
# highest params for food-101 dataset
# (0, 2, 0.5)
hog_train_model = svm_train(train_df['label'].tolist(), hog_train_feature, '-s 0 -t %s -g %s -c %s -b 1' % (high_params[0], high_params[1], high_params[2]))
svm_save_model('hog_food_classification.model', hog_train_model)
print('Training finished.')

### Testing

In [ ]:
# load json file
test_json = load_json(data_dir + '/meta/train.json')
# Create dataframe for test
test_df = create_data(test_json)

# Convert img to HOG nparray
hog_test_feature = HOG_training(test_df)

# Load in SVM model
hog_model = smv_load_model('hog_food_classification.model')

# Predict
hog_p_label, hog_p_acc, hog_p_val = svm_predict(test_df['label'].tolist(), hog_test_feature, hog_model, '-b 0')
hog_acc, hog_mse, hog_scc = evaluations(test_df['label'].tolist(), hog_p_label)

## Transfer Learning approach:

### Data processing

In [6]:
# Load image through the new model to get features 
def create_feature(data_df, pre_model):
    x_tmp = []
    size = len(data_df['filename'])
    print("total image: ", size)
    for i in range(0,size):
        image = data_df['filename'][i]
        label = data_df['label'][i]
        path = data_dir + '/images/' + get_key(label) + '/' + image
        #image_np = get_image(path)
        image_np = load_img(path, target_size=(227,227))
        image_np = img_to_array(image_np)
        image_np = np.expand_dims(image_np,axis=0)
        image_np = imagenet_utils.preprocess_input(image_np)
        x_tmp.append(image_np)
    x = np.vstack(x_tmp)
    features = pre_model.predict(x, batch_size=32)
    last_layer = pre_model.layers[-1].output_shape
    print("Reshape:", features.shape[0], last_layer[1] * last_layer[2] * last_layer[3])
    features_flatten = features.reshape((features.shape[0], last_layer[1] * last_layer[2] * last_layer[3]))
    return features_flatten

# Take in model hdf5 model and create a new model fpr transfer learning
def transfer_learning(train_df):
    # Load in hdf5 model
    # model name can be anything and can be change
    # i have mine as alexNet_best_weights.hdf5
    model = tf.keras.models.load_model(model_dir + '/alexNet_best_weigths.hdf5')
    # copy up to last convolutional layers
    drop_layer = 0
    for i in range(len(model.layers)-1, 0, -1):
        name = model.layers[i].name.split('_')[0]
        if(name == 'flatten'):
            drop_layer+= 1
            break
        drop_layer += 1
    new_model = tf.keras.Sequential()
    for layer in model.layers[:-drop_layer]:
        new_model.add(layer)
    new_model.trainable = False
    new_model.build((227,227,3))
    new_model.summary()
    train_feature = create_feature(train_df, new_model)
    return train_feature, new_model

### Training:

In [ ]:

# If you didn't run the HOG approach
#===============Uncomment this block==================
# # load json file
# train_json = load_json(data_dir + '/meta/train.json')
# # Create dataframe
# train_df = create_data(train_json)
#==================End block===========================

tl_train_feature, new_model = transfer_learning(train_df)


#====Uncoment for Grid search==========
#high_params = grid_search(train_df, tl_train_feature)
#===========End block==================

# b = 1 will give prob est
# highest params for food-101 dataset
# (0, 2, 0.5)
tl_train_model = svm_train(train_df['label'].tolist(), tl_train_feature, '-s 0 -t %s -g %s -c %s -b 1' % (high_params[0], high_params[1], high_params[2]))
svm_save_model('tl_food_classification.model', tl_train_model)
print('Training finished.')


# Grid Search
# Use small dataset to make search much faster
# Transfer Learning
# highest_params = grid_search(train_df, tl_train_feature)
# HOG
# highest_params = grid_search(train_df, hog_train_feature)

### Testing

In [ ]:
# In case you didn't run HOG test
#==============Uncomment this block===================
# # load json file
# test_json = load_json(data_dir + '/meta/train.json')
# # Create dataframe for test
# test_df = create_data(test_json)
#===================End block=========================

# Load in SVM model
tl_model = smv_load_model('tl_food_classification.model')

# Convert test images to features array
tl_test_feature = create_feature(test_df, tl_model)


# Predict
tl_p_label, tl_p_acc, tl_p_val = svm_predict(test_df['label'].tolist(), tl_test_feature, tl_model, '-b 0')
tl_acc, tl_mse, tl_scc = evaluations(test_df['label'].tolist(), tl_p_label)